In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\basde\OneDrive\Documenten\GitHub\Code-and-examples\Projects\Extrovert vs. Introvert Behavior Data\playground-series-s5e7\train.csv")
test = pd.read_csv(r"C:\Users\basde\OneDrive\Documenten\GitHub\Code-and-examples\Projects\Extrovert vs. Introvert Behavior Data\playground-series-s5e7\test.csv")

In [3]:
df = data.copy()
dft = test.copy()

In [4]:
observed_relationship = pd.crosstab(
    dft['Stage_fear'],
    dft['Drained_after_socializing'],
    dropna=True
)

print("Observed relationship between the two features:")
print(observed_relationship)

# We can also check the value counts for the non-null data to confirm the dominant pairs.
print("\nValue counts of non-null pairs:")
print(dft[['Stage_fear', 'Drained_after_socializing']].dropna().value_counts(normalize=True))

Observed relationship between the two features:
Drained_after_socializing    No   Yes
Stage_fear                           
No                         4055     3
Yes                           6  1098

Value counts of non-null pairs:
Stage_fear  Drained_after_socializing
No          No                           0.785548
Yes         Yes                          0.212708
            No                           0.001162
No          Yes                          0.000581
Name: proportion, dtype: float64


In [5]:
df1 = df.drop(['id','Stage_fear','Drained_after_socializing'], axis=1)
dft1 = dft.drop(['id','Stage_fear','Drained_after_socializing'], axis=1)

In [6]:
dft.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,6175.000000,5750.000000,5778.000000,5709.000000,5825.000000,5767.000000
mean,21611.000000,3.116870,5.287989,4.037835,8.008412,5.028958
std,1782.713288,2.985658,2.758052,2.045207,4.192701,2.867285
min,18524.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20067.500000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,21611.000000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,23154.500000,4.000000,8.000000,6.000000,12.000000,7.000000
max,24698.000000,11.000000,10.000000,7.000000,15.000000,10.000000


In [7]:
dft1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Time_spent_Alone         5750 non-null   float64
 1   Social_event_attendance  5778 non-null   float64
 2   Going_outside            5709 non-null   float64
 3   Friends_circle_size      5825 non-null   float64
 4   Post_frequency           5767 non-null   float64
dtypes: float64(5)
memory usage: 241.3 KB


In [8]:
dft1.describe()

,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,5750.000000,5778.000000,5709.000000,5825.000000,5767.000000
mean,3.116870,5.287989,4.037835,8.008412,5.028958
std,2.985658,2.758052,2.045207,4.192701,2.867285
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,3.000000,5.000000,3.000000
50%,2.000000,5.000000,4.000000,8.000000,5.000000
75%,4.000000,8.000000,6.000000,12.000000,7.000000
max,11.000000,10.000000,7.000000,15.000000,10.000000


In [9]:
from sklearn.impute import KNNImputer

In [10]:
# imputer = KNNImputer(n_neighbors=10)

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [12]:
imputer = IterativeImputer(max_iter=10, random_state=42)

In [13]:
dft1 = pd.DataFrame(imputer.fit_transform(dft1), columns=dft1.columns)

In [15]:
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.utils import resample
import numpy as np

# --- 1. Generate Diverse Base Partitions ---
# We create a list to hold all the individual clustering results
base_partitions = []
n_ensemble_members = 30 # Total number of models in our ensemble (10 of each type)
n_samples = dft1.shape[0]

# Generate partitions using different algorithms on bootstrap samples
# This creates the diversity needed for a good ensemble.
for i in range(n_ensemble_members):
    # Create a bootstrap sample (sampling with replacement)
    bootstrap_sample_indices = resample(range(n_samples), n_samples=n_samples, random_state=i)
    bootstrap_data = dft1.iloc[bootstrap_sample_indices]
    
    # Choose an algorithm based on the iteration
    if i % 3 == 0:
        # Use KMeans with a random number of clusters (2 or 3)
        n_k = np.random.randint(2, 4)
        model = KMeans(n_clusters=n_k, random_state=i, n_init='auto')
    elif i % 3 == 1:
        # Use Agglomerative Clustering
        model = AgglomerativeClustering(n_clusters=2)
    else:
        # Use Birch
        model = Birch(n_clusters=2)
        
    # Fit the model and store the labels
    labels = model.fit_predict(bootstrap_data)
    
    # We need to map the labels from the bootstrap sample back to the original data's indices
    partition = np.full(n_samples, -1, dtype=int)
    partition[bootstrap_sample_indices] = labels
    base_partitions.append(partition)

# --- 2. Build the Co-Association Matrix ---
# This matrix will store the similarity between each pair of data points.
co_association_matrix = np.zeros((n_samples, n_samples))

for partition in base_partitions:
    # Get indices of points that were actually clustered (not -1)
    active_indices = np.where(partition != -1)[0]
    
    # For each pair of points, if they are in the same cluster, increment their matrix entry
    for i in range(len(active_indices)):
        for j in range(i, len(active_indices)):
            idx1 = active_indices[i]
            idx2 = active_indices[j]
            if partition[idx1] == partition[idx2]:
                co_association_matrix[idx1, idx2] += 1
                co_association_matrix[idx2, idx1] += 1 # Matrix is symmetric

# Normalize the matrix to get similarity scores between 0 and 1
# This becomes our final similarity matrix for the last clustering step.
similarity_matrix = co_association_matrix / n_ensemble_members

# --- 3. Perform Final Clustering ---
# Use Agglomerative Clustering with the precomputed similarity matrix.
# This algorithm is ideal for grouping items based on a similarity graph.
final_clustering = AgglomerativeClustering(
    n_clusters=2,
    metric='precomputed',
    linkage='average' # 'average' linkage is often a good choice for this method
)

ensemble_labels = final_clustering.fit_predict(1 - similarity_matrix) # Needs dissimilarity

# --- 4. Assign Final Labels ---
dft1['Personality'] = ensemble_labels

# The cluster labels (0 and 1) are arbitrary. We need to assign them logically.
# We'll assume the larger cluster corresponds to 'Extrovert', which is a common pattern.
extrovert_label = dft1['Personality'].value_counts().idxmax()
introvert_label = 1 - extrovert_label

dft1['Personality'] = dft1['Personality'].map({
    extrovert_label: 'Extrovert',
    introvert_label: 'Introvert'
})

In [16]:
dft1.describe()

,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,6175.000000,6175.000000,6175.000000,6175.000000,6175.000000
mean,3.104766,5.243001,4.017216,7.968021,4.986169
std,2.935441,2.733332,2.015676,4.143302,2.833781
min,-0.546451,0.000000,0.000000,0.000000,-0.070486
25%,1.000000,3.000000,3.000000,5.000000,3.000000
50%,2.000000,5.000000,4.000000,8.000000,5.000000
75%,4.000000,7.113980,6.000000,11.000000,7.000000
max,11.000000,10.000000,7.000000,15.000000,10.000000


In [17]:
dft1['Personality'].value_counts()

Personality
Extrovert    4617
Introvert    1558
Name: count, dtype: int64

In [18]:
submission_df = pd.DataFrame({
    'id': dft['id'],
    'Personality': dft1['Personality']
})

In [19]:
submission_df.to_csv('submission.csv', index=False)

In [22]:
# Personality
# Extrovert    4619
# Introvert    1556
# Name: count, dtype: int64